In [2]:
import requests
import time
import pandas as pd
from sqlalchemy import create_engine

In [3]:
host = ""
port = ""
user = ""
password = ""
database= ""

### Messwerte der einzelnen Stationen in ein Dataframe speichern

In [ ]:
#Messstationen definieren
boxes_dic = {
    "Mersch Luxembourg" : "5ae4a726223bd80019a367a6",
    "Ettlingen Deutschland" : "5dde9523ba7944001da4150e",
    "Stuttgart Deutschland" : "5c08379b1c28f9001a3f580c",
    "München Deutschland" : "5b4d11485dc1ec001b5452c7",
    "Berlin Deutschland" : "65e8d93acbf5700007f920ca",
    "Hamburg Deutschland" : "590e0b0a51d3460011c725c4",
    "Dortmund Deutschland" : "605f498077a88b001bba3dc0",
    "Jena Deutschland" : "64722d1c9be0580007f776d9"
}

In [15]:
# Erstelle ein leeres DataFrame mit Spaltennamen
df_messwerte = pd.DataFrame(columns=["Time", "Temperature","Luftfeuchtigkeit", "PM10", "PM2.5", "senseID"])

In [16]:
new_data = []  # Eine leere Liste, um neue Daten zu sammeln
for box_id, box_value in boxes_dic.items():
    response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
    box = response.json() 
    temperature_value = None
    pm10_value = None
    pm25_value = None
    rel_Luftfeuchte_value = None

    # Traverse durch die 'sensors' Liste
    for sensor in box.get('sensors', []):
    # Überprüfe, ob der Sensor für Temperatur ist
        if sensor.get('title') == 'Temperatur':
           # Zugriff auf den Temperaturwert
            temperature_value = sensor.get('lastMeasurement', {}).get('value')
        
        if sensor.get('title') == 'PM10':
           # Zugriff auf den Temperaturwert
            pm10_value = sensor.get('lastMeasurement', {}).get('value')

        if sensor.get('title') == 'PM2.5':
           # Zugriff auf den Temperaturwert
            pm25_value = sensor.get('lastMeasurement', {}).get('value')

        if sensor.get('title') == 'rel. Luftfeuchte':
           # Zugriff auf den Temperaturwert
            rel_Luftfeuchte_value = sensor.get('lastMeasurement', {}).get('value')
            
            break  # Verlasse die Schleife, sobald die Temperatur gefunden ist
        
    # Füge neue Daten zur Liste hinzu
    new_data.append({"Temperature": temperature_value, "Time": time.strftime("%Y-%m-%d %H:%M:%S"),"Luftfeuchtigkeit": rel_Luftfeuchte_value, "PM10":pm10_value, "PM2.5":pm25_value, "senseID": box_value})
    
    # Konvertiere die neue Datenliste in ein DataFrame
new_df = pd.DataFrame(new_data)
    
    # Hänge das neue DataFrame an das bestehende DataFrame an
df_messwerte = pd.concat([df_messwerte, new_df], ignore_index=True)



In [17]:
df_messwerte

,Time,Temperature,Luftfeuchtigkeit,PM10,PM2.5,senseID
0,2024-05-14 11:23:31,23.50,86.30,6.90,1.67,5ae4a726223bd80019a367a6
1,2024-05-14 11:23:32,24.93,49.40,2.88,1.95,5dde9523ba7944001da4150e
2,2024-05-14 11:23:32,39.40,32.00,1.40,1.10,5c08379b1c28f9001a3f580c
3,2024-05-14 11:23:32,17.12,67.92,10.77,3.38,5b4d11485dc1ec001b5452c7
4,2024-05-14 11:23:33,26.50,31.10,None,None,65e8d93acbf5700007f920ca
5,2024-05-14 11:23:33,41.80,38.00,3.73,1.47,590e0b0a51d3460011c725c4
6,2024-05-14 11:23:34,26.0,None,None,None,605f498077a88b001bba3dc0
7,2024-05-14 11:23:34,23.96,39.02,3.33,1.67,64722d1c9be0580007f776d9


### Zweites DataFrame für Daten der Messstationen erstellen

In [11]:
# Erstelle ein leeres DataFrame mit Spaltennamen
df_stations = pd.DataFrame(columns=["Ort","Land","Lat", "Lon", "Hoehe", "senseID"])

In [12]:
new_data = []  # Eine leere Liste, um neue Daten zu sammeln
for box_id, box_value in boxes_dic.items():
    response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
    box = response.json() 
    temperature_value = None
    lat = None
    lon = None
    hoehe = None
    pm10_value = None
    pm25_value = None
    rel_Luftfeuchte_value = None
    ort_Land = box_id.split()

    # Extrahiere Koordinaten
    current_location = box.get('currentLocation', {})
    if current_location:
        coordinates = current_location.get('coordinates')
        if coordinates:
            lon = coordinates[0]
            lat = coordinates[1]
            height = coordinates[2] if len(coordinates) >= 3 else None

        
    # Füge neue Daten zur Liste hinzu
    new_data.append({"Ort": ort_Land[0], "Land":ort_Land[1], "Lat": lat,"Lon":lon,"Hoehe":height, "senseID": box_value})
    
    # Konvertiere die neue Datenliste in ein DataFrame
new_df = pd.DataFrame(new_data)
    
    # Hänge das neue DataFrame an das bestehende DataFrame an
df_stations = pd.concat([df_stations, new_df], ignore_index=True)

C:\Users\mariu\AppData\Local\Temp\ipykernel_13068\4227955529.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_stations = pd.concat([df_stations, new_df], ignore_index=True)


In [13]:
df_stations

,Ort,Land,Lat,Lon,Hoehe,senseID
0,Mersch,Luxembourg,49.754663,6.105682,250.0,5ae4a726223bd80019a367a6
1,Ettlingen,Deutschland,48.926209,8.374409,4.0,5dde9523ba7944001da4150e
2,Stuttgart,Deutschland,48.779610,9.213723,280.0,5c08379b1c28f9001a3f580c
3,München,Deutschland,48.146546,11.576033,20.0,5b4d11485dc1ec001b5452c7
4,Berlin,Deutschland,52.511270,13.397356,45.0,65e8d93acbf5700007f920ca
5,Hamburg,Deutschland,53.544046,9.997499,NaN,590e0b0a51d3460011c725c4
6,Dortmund,Deutschland,51.497350,7.494606,NaN,605f498077a88b001bba3dc0
7,Jena,Deutschland,50.934561,11.589701,146.0,64722d1c9be0580007f776d9
